##  Sentinel-2 burned area identification

This notebook prepares the data transformation application: The outcome is a data transformation application that takes one input (or a set of inputs organized in an atomic unit) and generates the output.

The application implements:

* Calculation of NDVI in the two scenes (using band 8 and 4), (B8-B4)/(B8+B4)
* Calculation of NDWI, in the two scenes (using band 8 and 11), (B8-B11)/(B8+B11)
* If NDWI i2 - NDWI i1 > 0.18 and If NDVI i2 - NDVI i1 > 0.19 then burned pixels

The outputs generated include:

* COG RGB composite with bands 12, 11, 8A pre
* COG RGB composite with bands 12, 11, 8A post 
* COG scene classification pre  
* COG scene classification post 
* COG 8 bits with bitmask burned/not burned
* Geojson with vectorization of bitmask burned/not burned

In [ ]:
service = dict([('title', 'Sentinel-2 burned area identification'),
                ('abstract', 'This is a short description'),
                ('id', 'ewf-satcen-03-03-02')])

### Parameter Definition 

### Runtime parameter definition

**Input reference**


In [ ]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                       ('value', '0.19'),
                       ('title', 'NDVI difference threshold'),
                       ('abstract', 'NDVI difference threshold'),
                       ('maxOccurs', '1')]) 

In [ ]:
ndwi_threshold = dict([('id', 'ndwi_threshold'),
                   ('value', '0.18'),
                   ('title', 'NDWI difference threshold'),
                   ('abstract', 'NDWI difference threshold'),
                   ('maxOccurs', '1')])

In [ ]:
wkt = dict([('id', 'aoi'),
            ('value', 'POLYGON ((150.4704663611788 -34.65618387777952, 150.4704663611788 -34.95618387777952, 150.1704663611788 -34.95618387777952, 150.1704663611788 -34.65618387777952, 150.4704663611788 -34.65618387777952))'),
            ('title', 'Area of interest'),
            ('abstract', 'Area of interest in WKT or bounding box')])

In [ ]:
input_references = ['https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191231T000241_N0213_R030_T56HKG_20191231T015159',
                   'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20191201T000241_N0213_R030_T56HKG_20191201T020044']

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data'

### Workflow

#### Import the packages

In [ ]:
import os
import sys
import cioppy
import snappy
from snappy import GPF
ciop = cioppy.Cioppy()
from snappy import jpy
from snappy import ProductIO
from datetime import datetime

import gdal
import geopandas as gp
import numpy as np
import datetime

sys.path.append(os.getcwd())
sys.path.append('/application/notebook/libexec/') 
from helpers import *

import warnings
warnings.filterwarnings('ignore')

gdal.UseExceptions() 

In [ ]:
products = get_metadata(input_references, data_path)

In [ ]:
group_analysis(products)

In [ ]:
products

In [ ]:
req_bands = ['B4','B8','B8A', 'B11','B12', 'quality_scene_classification' ]

In [ ]:
#if more than one post or pre products==> use slice assembly ==>use new mosaics as input to the next steps

In [ ]:
output_files = []
for index, item in enumerate(['Pst','Pre']):
    if(products[products['ordinal_type'] == item].identifier.count()>1):
        product = mosaic_inputs(products[products['ordinal_type'] == item].reset_index(drop=True))
    else:
        local_pathx=products[products['ordinal_type'] == item].iloc[0]['local_path']
        s2prd = "%s/MTD_MSIL2A.xml" %local_pathx 
        product = snappy.ProductIO.readProduct(s2prd)
    
    output_name = '%s_%s.tif'%(item,product.getName())
    #print(output_name)
    product = resample2ref_band(product,'B4')
    product = subset_to_aoi_reduce_bands(product,wkt['value'],req_bands)
    ProductIO.writeProduct(product, 'S2_{}_tmp.tif'.format(item), 'GeoTIFF-BigTIFF')
    output_files.append('RGB_{}'.format(output_name))
    snap_rgb(product,['B12','B11','B8A'],'RGB_{}'.format(output_name))
    output_files.append('MASK_{}'.format(output_name))
    snap_mask(product,'MASK_{}'.format(output_name))
print(output_files)

### NDVI & NDWI computation 

In [ ]:
ds = gdal.Open('S2_Pre_tmp.tif')
    
pre_b04 = ds.GetRasterBand(1).ReadAsArray()
pre_b08 = ds.GetRasterBand(2).ReadAsArray()
pre_b11 = ds.GetRasterBand(4).ReadAsArray()
pre_scl = ds.GetRasterBand(6).ReadAsArray()

ds = None

os.remove('S2_Pre_tmp.tif')

In [ ]:
ds = gdal.Open('S2_Pst_tmp.tif')
    
post_b04 = ds.GetRasterBand(1).ReadAsArray()
post_b08 = ds.GetRasterBand(2).ReadAsArray()
post_b11 = ds.GetRasterBand(4).ReadAsArray()
post_scl = ds.GetRasterBand(6).ReadAsArray()

width = ds.RasterXSize
height = ds.RasterYSize

input_geotransform = ds.GetGeoTransform()
input_georef = ds.GetProjectionRef()
#print(input_georef)
proj = osr.SpatialReference(wkt=ds.GetProjection())
espg = proj.GetAttrValue('AUTHORITY',1)
print(espg) 
ds = None

os.remove('S2_Pst_tmp.tif')

In [ ]:
gain = 10000

 ### NDWI with NIR (8) and SWIR (11)

In [ ]:
pre_ndwi2 = (pre_b08 / gain - pre_b11 / gain) / (pre_b08/ gain  + pre_b11 / gain)
post_ndwi2 = (post_b08 / gain - post_b11 / gain) / (post_b08/ gain  + post_b11 / gain)

pre_b11 = None
post_b11 = None

### NDVI with NIR (8) and Red (4)

In [ ]:
pre_ndvi = (pre_b08 / gain - pre_b04 / gain) / (pre_b08 / gain  + pre_b04 / gain)
post_ndvi = (post_b08 / gain - post_b04 / gain) / (post_b08 / gain  + post_b04 / gain)

In [ ]:
pre_b04 = None
post_b04 = None

pre_b08 = None
post_b08 = None

In [ ]:
conditions = (((post_ndwi2 - pre_ndwi2)  > float(ndwi_threshold['value'])) & ((post_ndvi - pre_ndvi) > float(ndvi_threshold['value'])) & (pre_scl == 4) | (post_scl == 4)) 

In [ ]:
burned = np.zeros((height, width), dtype=np.uint8)

In [ ]:
burned[conditions] = 1

In [ ]:
pre_ndwi2 = None
post_ndwi2 = None

pre_ndvi = None
post_ndvi = None

### Exclude according to scene classifications:

where noData put burned=2 if burn then put burned=1 else burned=0

In [ ]:
burned[np.where((pre_scl == 0) | (post_scl == 0) | (pre_scl == 1) | (post_scl == 1) | (pre_scl == 5) | (post_scl == 5) | (pre_scl == 6) | (post_scl == 6) | (pre_scl == 7) | (post_scl == 7) | (pre_scl == 8) | (post_scl == 8) | (pre_scl == 9) | (post_scl == 9))] = 2

Write

In [ ]:
start_date=(products[products['ordinal_type'] == 'Pre'].reset_index(drop=True)).iloc[0].startdate[0:10]

In [ ]:
end_date=(products[products['ordinal_type'] == 'Pst'].reset_index(drop=True)).iloc[0].enddate[0:10]

In [ ]:
#Requested file name : 'Burned_Area_S2_{MasterId}_{SlaveId}.tif
if products[products['ordinal_type'] == 'Pre'].identifier.count() == 1 and products[products['ordinal_type'] == 'Pst'].identifier.count() == 1:
    output_name_Burned_Area = 'Burned_Area_S2_%s_%s.tif'%(products[products['ordinal_type'] == 'Pst'].iloc[0]['identifier'],products[products['ordinal_type'] == 'Pre'].iloc[0]['identifier'])
else:
    #if inputs are mosaiced 
    output_name_Burned_Area = 'Burned_Area_S2_%s_%s.tif'%(end_date,start_date)

In [ ]:
write_tif(burned, output_name_Burned_Area, width, height, input_geotransform, input_georef)
output_files.append(output_name_Burned_Area)

In [ ]:
change_detection_gp = polygonize(output_name_Burned_Area, 1,espg )

In [ ]:
output_files.append('polygonized.json')

#### if we replace {'init':'epsg:{}'.format(epsg)} with new recommended 'epsg:{}', the axis order changes

In [ ]:
change_detection_gp.head(10)

### Get the result WKT

In [ ]:
src = gdal.Open(output_name_Burned_Area)
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

max_x = ulx + (src.RasterXSize * xres)
min_y = uly + (src.RasterYSize * yres)
min_x = ulx 
max_y = uly

min_x, min_y, max_x, max_y

In [ ]:
source = osr.SpatialReference()
source.ImportFromWkt(src.GetProjection())

target = osr.SpatialReference()
target.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(source, target)

result_wkt = box(transform.TransformPoint(min_x, min_y)[0],
        transform.TransformPoint(min_x, min_y)[1],
        transform.TransformPoint(max_x, max_y)[0],
        transform.TransformPoint(max_x, max_y)[1]).wkt

In [ ]:
result_wkt

### Create the properties file

In [ ]:
from datetime import datetime

In [ ]:
products.sort_values(by='startdate',ascending=True,inplace=True)

masterID = products.iloc[0].identifier
slaveID = products.iloc[1].identifier

In [ ]:
date_format = '%Y-%m-%dT%H:%m:%SZ'

In [ ]:
for index , item in enumerate(output_files):

    if 'RGB' in item:
        prod = slaveID
        if 'Pre' in item[4:7]:
            prod = masterID
        title = 'Sentinel-2 RGB {}-event {} (B11, B12, B8A)'.format(item[4:7],prod)
            
    if 'MASK' in item:
        prod = slaveID
        if 'Pre' in item[5:8]:
            prod = masterID
        title = 'Sentinel-2 Scene Classification {}-event {}'.format(item[5:8],prod)

    if 'Burned_Area_S2' in item:
        title = 'Sentinel-2 burned area identification for pair {}/{}'.format(masterID,slaveID)
    
    if 'polygonized' in item:
        title = 'Geojson with vectorization of bitmask burned=1/not burned=0/unkown=2 for pair {}/{}'.format(masterID,slaveID)
        
    
    with open('{}.properties'.format(item), 'wb') as file:
        file.write('title={}\n'.format(title))
        if 'Pre-event' in title:
            start_date_iso = pd.to_datetime(products.iloc[0].startdate).strftime(date_format)
            end_date_iso = pd.to_datetime(products.iloc[0].enddate).strftime(date_format)
            file.write('date={}/{}\n'.format(start_date_iso,start_date_iso))
        elif 'Pst-event' in title :
            start_date_iso = pd.to_datetime(products.iloc[1].startdate).strftime(date_format)
            end_date_iso = pd.to_datetime(products.iloc[1].enddate).strftime(date_format)
            file.write('date={}/{}\n'.format(end_date_iso,end_date_iso))
        else:
            start_date_iso = pd.to_datetime(products.iloc[0].startdate).strftime(date_format)
            end_date_iso = pd.to_datetime(products.iloc[1].enddate).strftime(date_format)
            file.write('date={}/{}\n'.format(start_date_iso,end_date_iso))
            
        file.write('geometry={}'.format(result_wkt))


### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.